# NLP LLM Classroom Demo

This notebook demonstrates a compact workflow for teaching natural language processing tasks with pretrained large language models (LLMs) from [Hugging Face](https://huggingface.co/models). Each section contains:

1. A short explanation of the task.
2. A runnable code cell using a ready-to-use model.
3. Discussion of the output and classroom talking points.

> **Note:** All examples rely on freely available, relatively small models so that they run quickly on Google Colab's default CPU runtime. Replace them with larger models for higher quality when more resources are available.

## 0. Environment Setup

Only two lightweight libraries are needed. `transformers` gives us the model pipelines and `sentencepiece` is required by a few multilingual models. The installation takes ~1 minute in Colab.

In [ ]:
%pip install -q transformers sentencepiece

## 1. Imports and Helper Utilities

The `pipeline` helper from 🤗 Transformers abstracts away tokenization, model loading, and post-processing. The helper function `run_pipeline` standardizes how we print model outputs for the classroom demo.

In [ ]:
from transformers import pipeline

def run_pipeline(task_pipeline, *args, **kwargs):
    '''Execute a pipeline call and pretty-print the result.'''
    result = task_pipeline(*args, **kwargs)
    if isinstance(result, list):
        for idx, item in enumerate(result, start=1):
            print(f"Result {idx}:")
            if isinstance(item, dict):
                for key, value in item.items():
                    print(f"  {key}: {value}")
            else:
                print(item)
            print()
    elif isinstance(result, dict):
        for key, value in result.items():
            print(f"{key}: {value}")
    else:
        print(result)
    return result

## 2. Sentiment Analysis

**Goal:** Classify the emotional tone (positive/negative) of a text snippet.

**Model:** `distilbert-base-uncased-finetuned-sst-2-english` (fine-tuned on the Stanford Sentiment Treebank).

In [ ]:
sentiment_pipeline = pipeline(
    task="sentiment-analysis",
    model="distilbert-base-uncased-finetuned-sst-2-english"
)

sample_reviews = [
    "The new course content is incredibly engaging!",
    "I struggled to understand the instructions and felt lost."
]

sentiment_results = run_pipeline(sentiment_pipeline, sample_reviews)
sentiment_results

### Teaching Notes
- Highlight how the model returns both the predicted label and confidence score (`score`).
- Encourage students to think about class imbalance and what kinds of text might be ambiguous for the classifier.
- Prompt a discussion about expanding from binary to multi-class sentiment analysis.

## 3. Named Entity Recognition (NER)

**Goal:** Detect and categorize real-world entities (people, organizations, locations, etc.) mentioned in text.

**Model:** `dslim/bert-base-NER`, a multilingual BERT fine-tuned for token-level entity tagging.

In [ ]:
ner_pipeline = pipeline(
    task="ner",
    model="dslim/bert-base-NER",
    aggregation_strategy="simple"
)

ner_text = (
    "Hugging Face was founded in New York and now has offices in Paris and San Francisco."
)

ner_results = run_pipeline(ner_pipeline, ner_text)
ner_results

### Teaching Notes
- Point out how entity spans are grouped with the `aggregation_strategy`. Without it, students would see token-level predictions.
- Ask learners to consider how the model might struggle with less common entity types or languages.

## 4. Extractive Question Answering

**Goal:** Given a question and a reference passage, extract the span of text that best answers the question.

**Model:** `distilbert-base-uncased-distilled-squad`, fine-tuned on the SQuAD dataset.

In [ ]:
qa_pipeline = pipeline(
    task="question-answering",
    model="distilbert-base-uncased-distilled-squad"
)

qa_context = (
    "Large language models are pretrained on enormous text corpora. "
    "Fine-tuning them on task-specific data can drastically improve accuracy."
)

qa_prompt = {
    "question": "Why do we fine-tune pretrained language models?",
    "context": qa_context
}

qa_result = run_pipeline(qa_pipeline, qa_prompt)
qa_result

### Teaching Notes
- Emphasize the difference between extractive QA (selecting text) and generative QA (producing text from scratch).
- Discuss scenarios where the answer is not present in the context and how the model might respond.

## 5. Abstractive Summarization

**Goal:** Compress long-form text into a concise summary that captures the main ideas.

**Model:** `sshleifer/distilbart-cnn-12-6`, a distilled version of BART fine-tuned on news articles. It is fast enough for live demos.

In [ ]:
summarization_pipeline = pipeline(
    task="summarization",
    model="sshleifer/distilbart-cnn-12-6"
)

long_text = (
    "Transformers have transformed NLP by enabling models to learn contextual relationships "
    "in text using self-attention. They underpin state-of-the-art systems for translation, "
    "summarization, and question answering, and are now being scaled into multi-billion "
    "parameter large language models deployed in production."
)

summary = run_pipeline(
    summarization_pipeline,
    long_text,
    max_length=60,
    min_length=25,
    do_sample=False
)
summary

### Teaching Notes
- Ask students to compare the generated summary with the original paragraph.
- Talk about length control via `max_length` and `min_length` and why deterministic decoding (`do_sample=False`) is helpful for reproducible demos.

## 6. Generative Text Completion (GPT-style)

**Goal:** Produce fluent continuations of a prompt using an auto-regressive language model.

**Model:** `distilgpt2`, a compact GPT-2 variant suitable for CPU demos. This illustrates the workflow behind larger GPT-family models (including future versions such as GPT-5).

In [ ]:
generation_pipeline = pipeline(
    task="text-generation",
    model="distilgpt2",
    pad_token_id=50256  # ensure the pipeline handles end-of-text token correctly
)

generation_prompt = "In the future of AI education, teachers will"

completion = run_pipeline(
    generation_pipeline,
    generation_prompt,
    max_length=80,
    num_return_sequences=1,
    temperature=0.7
)
completion

### Teaching Notes
- Explain sampling parameters like `temperature` and `num_return_sequences`.
- Encourage experimentation with different prompts to explore model creativity and limitations.
- Connect this demo to production-scale GPT workflows (e.g., GPT-4 or future GPT-5) that follow the same API pattern but require managed inference endpoints.

## 7. (Optional) Connecting to Hosted LLMs

If you have access to managed endpoints (e.g., Hugging Face Inference Endpoints or future GPT-5 APIs), you can adapt the same workflow:

```python
from transformers import pipeline

hf_hub_pipeline = pipeline(
    "text-generation",
    model="HuggingFaceH4/zephyr-7b-beta",
    token="YOUR_HF_TOKEN",
    revision="main"
)
print(hf_hub_pipeline("Explain the role of attention in transformers."))
```

For OpenAI-style GPT endpoints, the logic is similar—submit a prompt and receive generated text—but requires the `openai` (or `openai-python`) package and an API key. Keeping this optional avoids extra dependencies for the classroom session.

## 8. Cleanup Tip

Models downloaded during the session are cached under `~/.cache/huggingface`. In shared environments, you can clear them after class with:

```python
!rm -rf ~/.cache/huggingface
```

This step is optional but keeps storage usage low on repeated runs.